## Sampling of Control Points
- **Input**: a raster file representing the study area, where the value of each pixel is assigned to one of 5 vegetation classes. These classes were obtained by partitioning the area using K-means clustering based on NDVI, MSAVI, albedo and TGSI values over the study period.
- **Output**: We sample an equal number of control points with an arbitrary date in each class. The resulting json and shapefile are used to generate the corresponding spectral indices feeded to the linear regression model.

In [1]:
from osgeo import gdal
import random
import json
import shapefile

Global variables

In [2]:
tiff_file = './rasters/clustered_study_area.tif'
out_json = './models/control-points.json'
out_shp_prefix = './models/control-points'

cp_number = 5000
cp_per_class = 1000
first_year = 2002
last_year = 2022

Open the raster file and get image dimensions

In [3]:
raster = gdal.Open(tiff_file)
xsize = raster.RasterXSize
ysize = raster.RasterYSize
num_pixels = xsize * ysize
num_bands = raster.RasterCount

In [4]:
print(f"the number of bands: {num_bands}")
print(f"The number of rows: {xsize}")
print(f"The number of columns: {ysize}")
print(f"The number of pixels in the image: {num_pixels}")

the number of bands: 1
The number of rows: 39675
The number of columns: 14808
The number of pixels in the image: 587507400


---
Get the CRS of the raster

In [5]:
crs = raster.GetProjection()
print(f"The CRS of the GeoTIFF file is: {crs}")

The CRS of the GeoTIFF file is: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]


The **[GetGeoTransform](https://gdal.org/tutorials/geotransforms_tut.html)** function returns a tuple with following attributes:\
- geo_tr(0) x-coordinate of the upper-left corner of the upper-left pixel.
- geo_tr(1) w-e pixel resolution / pixel width.
- geo_tr(2) row rotation (typically zero).
- geo_tr(3) y-coordinate of the upper-left corner of the upper-left pixel.
- geo_tr(4) column rotation (typically zero).
- geo_tr(5) n-s pixel resolution / pixel height (negative value for a north-up image).

In [6]:
# Get the geo-transform matrix
geo_tr = raster.GetGeoTransform()
print(f"Coordinates of the upper-left corner of the upper-left pixel: {geo_tr[0]:.5f},{geo_tr[3]:.5f}")
print(f"Pixel size: {geo_tr[1]:.5f},{geo_tr[5]:.5f}")

Coordinates of the upper-left corner of the upper-left pixel: -2.21551,36.04490
Pixel size: 0.00027,-0.00027


Read the raster as an array

In [7]:
band_data = raster.GetRasterBand(1)
pixel_array = band_data.ReadAsArray()

This function takes the coordinates of a pixel in the raster and returns its geographic coordinates

In [8]:
def geo_coord(x,y):
    x_origin = geo_tr[0]
    y_origin = geo_tr[3]
    pixel_width = geo_tr[1]
    pixel_height = geo_tr[5]
    x_geo = x_origin + x * pixel_width + (pixel_width / 2)
    y_geo = y_origin + y * pixel_height + (pixel_height / 2)
    return (x_geo,y_geo)

An example of the calculation of the geographical coordinates of a pixel

In [9]:
pixel_x = 7400  
pixel_y = 19000 
(x_geo,y_geo) = geo_coord(pixel_x,pixel_y)
print(f"The geographical coordinates of pixel ({pixel_x},{pixel_y}) are ({x_geo:.5f}, {y_geo:.5f})")

The geographical coordinates of pixel (7400,19000) are (-0.22112, 30.92437)


---
The following loop performs a stratified sampling of control points to represent the 5 vegetation classes over the study area and returns a json file. A random date within the study period is given to attributed to each point.

In [10]:
iter = 0
current_samples_per_class = [0,0,0,0,0]
samples_list = []
while iter < cp_number: 
    rnd_pixel_x= random.randint(0, xsize-1)
    rnd_pixel_y = random.randint(0, ysize-1)
    rnd_pixel_val = pixel_array[rnd_pixel_y,rnd_pixel_x]
    if rnd_pixel_val != 0 and current_samples_per_class[rnd_pixel_val-1]<cp_per_class :
        current_samples_per_class[rnd_pixel_val-1] += 1
        geo_coord_pixel = geo_coord(rnd_pixel_x,rnd_pixel_y)        
        random_date = "{:04d}".format(random.randint(first_year, last_year))+'-'+"{:02d}".format(random.randint(1, 12))+'-'+"{:02d}".format(random.randint(1, 28))
        samples_list.append({'latitude': str(geo_coord_pixel[1]), 'longitude': str(geo_coord_pixel[0]), 
                          'class': str(rnd_pixel_val),'map_date': random_date})
        iter += 1
print(f"Number of samples per class: {current_samples_per_class}")
with open(out_json, 'w') as json_file:
    json.dump(samples_list, json_file)

Number of samples per class: [1000, 1000, 1000, 1000, 1000]


A shapefile of the generated control points is also generated

In [11]:
 # Define the shapefile attributes
fields = [("ID", "N", 6, 0), ("Class", "N", 1, 0),("map_date", "C", 10)]

# Define the shapefile writer
w = shapefile.Writer(out_shp_prefix, shapeType=shapefile.POINT)
for field in fields:
    w.field(*field)
    
id = 1 
for point in samples_list:
    w.point(float(point["longitude"]), float(point["latitude"]))
    w.record(id,point['class'],point["map_date"])
    id += 1
w.crs = crs
# Save the shapefile
w.close()